In [ ]:
import kagglehub
path = kagglehub.dataset_download("ascanipek/eyepacs-aptos-messidor-diabetic-retinopathy")

Using Colab cache for faster access to the 'eyepacs-aptos-messidor-diabetic-retinopathy' dataset.


In [ ]:
import os
print(path)
print(os.listdir(path))

/kaggle/input/eyepacs-aptos-messidor-diabetic-retinopathy
['dr_unified_v2', 'augmented_resized_V2']


In [ ]:
# tf.__version__

In [ ]:
# ===============================================================
#   EfficientNet-B4 DR Severity Classifier
#   Memory-Stable + Macro F1/Precision/Recall + QWK Metric
# ===============================================================

import os, gc, json
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import mixed_precision

# ---------------------------------------------------------------
# CONFIG
# ---------------------------------------------------------------
DATASET_DIR = '/kaggle/input/eyepacs-aptos-messidor-diabetic-retinopathy/augmented_resized_V2'
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR   = os.path.join(DATASET_DIR, "val")
TEST_DIR  = os.path.join(DATASET_DIR, "test")

IMG_SIZE = (300, 300)
BATCH_SIZE = 8
SHUFFLE_BUF = 256
NUM_CLASSES = 5
SEED = 42

EPOCHS_STAGE1 = 4
EPOCHS_STAGE2 = 4
EPOCHS_STAGE3 = 8

LR1 = 1e-3
LR2 = 5e-5
LR3 = 1e-5

WEIGHTS_BEST = "b4_best.weights.h5"
WEIGHTS_FINAL = "b4_final.weights.h5"
ARCH_JSON = "b4_model_architecture.json"

# ---------------------------------------------------------------
# Mixed Precision
# ---------------------------------------------------------------
mixed_precision.set_global_policy("mixed_float16")
print("Mixed precision:", mixed_precision.global_policy())

# ---------------------------------------------------------------
# Class Weights
# ---------------------------------------------------------------
class_names = sorted(os.listdir(TRAIN_DIR))
class_counts = {c: len(os.listdir(os.path.join(TRAIN_DIR, c))) for c in class_names}
print("Class counts:", class_counts)

labels_list = []
for idx, c in enumerate(class_names):
    labels_list += [idx] * class_counts[c]

cw = compute_class_weight("balanced", classes=np.unique(labels_list), y=labels_list)
class_weights = {i: float(cw[i]) for i in range(len(cw))}
print("Class weights:", class_weights)

# ---------------------------------------------------------------
# Dataset Loading
# ---------------------------------------------------------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN_DIR,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=True,
    seed=SEED,
    label_mode="categorical"
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    VAL_DIR,
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,
    label_mode="categorical"
)

test_ds = None
if os.path.exists(TEST_DIR):
    test_ds = tf.keras.utils.image_dataset_from_directory(
        TEST_DIR,
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        shuffle=False,
        label_mode="categorical"
    )

AUTOTUNE = tf.data.AUTOTUNE

# ---------------------------------------------------------------
# Preprocessing
# ---------------------------------------------------------------
from tensorflow.keras.applications.efficientnet import preprocess_input

def preprocess(images, labels):
    images = tf.cast(images, tf.float32)
    images = preprocess_input(images)
    return images, labels

train_ds = train_ds.shuffle(SHUFFLE_BUF).map(preprocess, AUTOTUNE).prefetch(AUTOTUNE)
val_ds   = val_ds.map(preprocess, AUTOTUNE).prefetch(AUTOTUNE)
if test_ds:
    test_ds = test_ds.map(preprocess, AUTOTUNE).prefetch(AUTOTUNE)

# ---------------------------------------------------------------
# Minimal Augmentation
# ---------------------------------------------------------------
def augment(images, labels):
    images = tf.image.random_flip_left_right(images)
    images = tf.image.random_brightness(images, 0.10)
    images = tf.image.random_contrast(images, 0.85, 1.15)
    return images, labels

train_ds = train_ds.map(augment, AUTOTUNE)

# ---------------------------------------------------------------
# Advanced Metrics (Lightweight)
# ---------------------------------------------------------------

class QWK_Metric(tf.keras.metrics.Metric):
    def __init__(self, num_classes=5, name="qwk", **kwargs):
        super().__init__(name=name, **kwargs)
        self.num_classes = num_classes
        self.cm = self.add_weight(
            name="cm",
            shape=(num_classes, num_classes),
            initializer="zeros",
            dtype=tf.float32
        )

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, 1)
        y_pred = tf.argmax(y_pred, 1)
        m = tf.math.confusion_matrix(
            y_true,
            y_pred,
            num_classes=self.num_classes,
            dtype=tf.float32
        )
        self.cm.assign_add(m)

    def result(self):
        cm = self.cm
        w = tf.zeros_like(cm)
        for i in range(self.num_classes):
            for j in range(self.num_classes):
                w = tf.tensor_scatter_nd_update(
                    w, [[i, j]], [(i - j) ** 2 / (self.num_classes - 1) ** 2]
                )
        act = tf.reduce_sum(cm, axis=1)
        pred = tf.reduce_sum(cm, axis=0)
        expected = tf.tensordot(act, pred, axes=0) / tf.reduce_sum(cm)
        return 1 - tf.reduce_sum(w * cm) / tf.reduce_sum(w * expected)

    def reset_state(self):
        self.cm.assign(tf.zeros_like(self.cm))


def macro_f1(y_true, y_pred):
    y_true = tf.argmax(y_true,1)
    y_pred = tf.argmax(y_pred,1)
    return tf.keras.metrics.f1_score(y_true, y_pred, average="macro")

def macro_precision(y_true, y_pred):
    return tf.keras.metrics.Precision(name="macro_precision")(y_true, y_pred)

def macro_recall(y_true, y_pred):
    return tf.keras.metrics.Recall(name="macro_recall")(y_true, y_pred)

# ---------------------------------------------------------------
# Model
# ---------------------------------------------------------------
def build_model():
    base = tf.keras.applications.EfficientNetB4(
        include_top=False, weights="imagenet",
        input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
    )

    inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3), dtype=tf.float32)
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.6)(x)                 # Increased dropout
    x = layers.Dense(512, activation="relu", dtype="float32")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(NUM_CLASSES, activation="softmax", dtype="float32")(x)
    return Model(inputs, outputs)

model = build_model()
print(model.summary())

loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.05)

metrics = [
    tf.keras.metrics.AUC(name="auc"),
    QWK_Metric(),
]

callbacks = [
    ModelCheckpoint(WEIGHTS_BEST, save_weights_only=True,
                    monitor="val_auc", mode="max", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.3, verbose=1),
    EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)
]

# ---------------------------------------------------------------
# TRAINING - Stage 1
# ---------------------------------------------------------------
for layer in model.layers[1].layers:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(LR1),
              loss=loss, metrics=metrics)

print("=== Stage 1 ===")
history1 = model.fit(train_ds, validation_data=val_ds,
                     epochs=EPOCHS_STAGE1,
                     class_weight=class_weights,
                     callbacks=callbacks)

gc.collect()

# ---------------------------------------------------------------
# TRAINING - Stage 2
# ---------------------------------------------------------------
model = build_model()
model.load_weights(WEIGHTS_BEST)

base = model.layers[1]
total_layers = len(base.layers)
start = int(total_layers * 0.7)

for i, layer in enumerate(base.layers):
    layer.trainable = (i >= start)

model.compile(optimizer=tf.keras.optimizers.Adam(LR2),
              loss=loss, metrics=metrics)

print("=== Stage 2 ===")
history2 = model.fit(train_ds, validation_data=val_ds,
                     epochs=EPOCHS_STAGE2,
                     class_weight=class_weights,
                     callbacks=callbacks)

gc.collect()

# ---------------------------------------------------------------
# TRAINING - Stage 3
# ---------------------------------------------------------------
model = build_model()
model.load_weights(WEIGHTS_BEST)

for layer in model.layers[1].layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(LR3),
              loss=loss, metrics=metrics)

print("=== Stage 3 ===")
history3 = model.fit(train_ds, validation_data=val_ds,
                     epochs=EPOCHS_STAGE3,
                     class_weight=class_weights,
                     callbacks=callbacks)

# ---------------------------------------------------------------
# SAVE MODEL
# ---------------------------------------------------------------
model.save_weights(WEIGHTS_FINAL)
print("Saved:", WEIGHTS_FINAL)

with open(ARCH_JSON, "w") as f:
    f.write(model.to_json())
print("Saved architecture:", ARCH_JSON)


Mixed precision: <DTypePolicy "mixed_float16">
Class counts: {'0': 55162, '1': 18470, '2': 24198, '3': 7936, '4': 9475}
Class weights: {0: 0.4178274899387259, 1: 1.2478722252301029, 2: 0.9524836763368874, 3: 2.904259072580645, 4: 2.432527704485488}
Found 115241 files belonging to 5 classes.
Found 14227 files belonging to 5 classes.
Found 14201 files belonging to 5 classes.


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 300, 300, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb4 (Functional)     │ (None, 10, 10, 1792)   │    17,673,823 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1792)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 1792)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 512)            │       918,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 5)              │         2,565 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,594,404 (70.93 MB)

 Trainable params: 18,469,197 (70.45 MB)

 Non-trainable params: 125,207 (489.09 KB)

None
=== Stage 1 ===
Epoch 1/4
14405/14406 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - auc: 0.7614 - loss: 1.3973 - qwk: 0.4777
Epoch 1: saving model to b4_best.weights.h5
14406/14406 ━━━━━━━━━━━━━━━━━━━━ 462s 27ms/step - auc: 0.7614 - loss: 1.3973 - qwk: 0.4777 - val_auc: 0.8763 - val_loss: 1.0948 - val_qwk: 0.6888 - learning_rate: 0.0010
Epoch 2/4
14406/14406 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - auc: 0.8085 - loss: 1.3058 - qwk: 0.5558
Epoch 2: saving model to b4_best.weights.h5
14406/14406 ━━━━━━━━━━━━━━━━━━━━ 281s 19ms/step - auc: 0.8085 - loss: 1.3058 - qwk: 0.5558 - val_auc: 0.8746 - val_loss: 1.1346 - val_qwk: 0.6918 - learning_rate: 0.0010
Epoch 3/4
14405/14406 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - auc: 0.8157 - loss: 1.2909 - qwk: 0.5592
Epoch 3: saving model to b4_best.weights.h5
14406/14406 ━━━━━━━━━━━━━━━━━━━━ 283s 20ms/step - auc: 0.8157 - loss: 1.2909 - qwk: 0.5592 - val_auc: 0.8728 - val_loss: 1.1250 - val_qwk: 0.6992 - learning_rate: 0.0010
Epoch 4/4
14406/14406 ━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1408506528.py", line 2, in <cell line: 0>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.12/dist-packages/google/colab/drive.py", line 97, in mount
    return _mount(
           ^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/drive.py", line 134, in _mount
    _message.blocking_request(
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_message.py", line 176, in blocking_request
    return read_reply_from_input(request_id, timeout_sec)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/google/colab/_message.py", line 96, in read_reply_from_input
    time.sleep(0.025)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (

TypeError: object of type 'NoneType' has no len()

In [ ]:
from google.colab import files

files_to_download = [
    WEIGHTS_BEST,
    WEIGHTS_FINAL,
    ARCH_JSON
]

for f in files_to_download:
    try:
        files.download(f)
        print(f"✔ Downloading {f}")
    except:
        print(f"✘ Could not download {f}")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Downloading b4_best.weights.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Downloading b4_final.weights.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Downloading b4_model_architecture.json
